In [15]:
import json
from collections import defaultdict
import re

input_file_name = 'Twitter5July_Data.txt'
retweet_file_name = 'TwitterRetweet.gml'
hashtag_file_name = "TwitterHashtags.gml"
MINIMUM_EDGE_WEIGHT = 6

In [2]:
### File Loading ###

tweets_texts = []
tweets_users = []
tweets_file = open(input_file_name, "r")
for line in tweets_file:
    try:
        tweet = json.loads(line)
        tweets_texts.append(tweet['text'].encode('utf-8'))
        tweets_users.append(tweet['user']['screen_name'])
    except:
        continue

In [3]:
### Writing Retweets to gml ###
output_file = open(retweet_file_name, "w")
output_file.write("graph\n")
output_file.write("[\n")

counter = 0
max_length = len(tweets_texts)
pairwise_counter_dictionary = defaultdict(int)
user_id = 0
users = {}
while counter < max_length:
    text = tweets_texts[counter]
    text = text.strip()
    text_tokens = text.split()

    from_user = tweets_users[counter]
    prev_token = ""
    for token in text_tokens:
        token = token.decode('utf-8')
        
        if prev_token == 'RT' and token.startswith('@'):
            #print (token)
            token = token.replace(":", "")
            to_user = token.replace("@", "")
            #print (from_user + ',' + to_user)

            from_user_id = ""
            to_user_id = ""
            if (from_user in users) == False:
                users[from_user] = user_id
                user_id = user_id + 1
            from_user_id = users[from_user]
            if (to_user in users) == False:
                users[to_user] = user_id
                user_id = user_id + 1
            to_user_id = users[to_user]

            # increment counter
            pairwise_counter_dictionary[from_user_id, to_user_id] += 1

        prev_token = token

    # while loop counter
    counter = counter + 1

# print all nodes
for key in users.keys():
    output_file.write("  node\n")
    output_file.write("  [\n")
    output_file.write("    id " + str(users[key]) + "\n")
    output_file.write("    label \"" + key + "\"" + "\n")
    output_file.write("  ]\n")

# print all edges
for keys in pairwise_counter_dictionary.keys():
    output_file.write("  edge\n")
    output_file.write("  [\n")
    output_file.write("    source " + str(keys[0]) + "\n")
    output_file.write("    target " + str(keys[1]) + "\n")
    output_file.write("    value " + str(pairwise_counter_dictionary[keys[0], keys[1]]) + "\n")
    output_file.write("  ]\n")

output_file.write("]\n")
output_file.close()
print("gml written")

gml written


In [16]:
### Writing Hashtags to gml ###
output_file = open(hashtag_file_name, "w", encoding= "utf-8")
output_file.write("graph\n")
output_file.write("[\n")
counter = 0
max_length = len(tweets_texts)
pairwise_counter_dictionary = defaultdict(int)
hashtags = {}
hashtag_id = 0
while counter < max_length:
    text = tweets_texts[counter]
    text = text.lower()
    text = text.strip()    
    text_tokens = text.split()
    
    hashtag_tokens = []
    for token in text_tokens:
        token = token.decode('utf-8')
        
        if token.startswith("#"):
            token = token.replace("#", "")
            hashtag_tokens.append(token)

    #print hashtag_tokens
    if len(hashtag_tokens) > 1:
        for i in range(0, len(hashtag_tokens)):
            for j in range((i + 1), len(hashtag_tokens)):
                #print ()"i: " + str(i) + ", j: " + str(j))
                left_id = ""
                right_id = ""
                if(hashtag_tokens[i] in hashtags.keys()) == False:
                    hashtags[hashtag_tokens[i]] = hashtag_id
                    hashtag_id = hashtag_id + 1
                left_id = hashtags[hashtag_tokens[i]]
                if(hashtag_tokens[j] in hashtags.keys()) == False:
                    hashtags[hashtag_tokens[j]] = hashtag_id
                    hashtag_id = hashtag_id + 1
                right_id = hashtags[hashtag_tokens[j]]
                #increment the counter
                if pairwise_counter_dictionary[left_id, right_id] == None:
                    if pairwise_counter_dictionary[right_id, left_id] == None:
                        pairwise_counter_dictionary[left_id, right_id] = 1
                    else:
                        pairwise_counter_dictionary[right_id, left_id] = pairwise_counter_dictionary[right_id, left_id] + 1
                else:
                    pairwise_counter_dictionary[left_id, right_id] = pairwise_counter_dictionary[left_id, right_id] + 1
    #print (pairwise_counter_dictionary)
    #exit(1)
    # while loop counter
    counter = counter + 1

# print all edges
# print only those whose edge weight > 3
keys_included = defaultdict(int)
for keys in pairwise_counter_dictionary.keys():
    if (pairwise_counter_dictionary[keys[0], keys[1]]) >= MINIMUM_EDGE_WEIGHT:
        keys_included[keys[0]] += 1
        keys_included[keys[1]] += 1

# print all nodes
for key in hashtags.keys():
    if hashtags[key] in keys_included.keys():
        output_file.write("  node\n")
        output_file.write("  [\n")
        output_file.write("    id " + str(hashtags[key]) + "\n")
        output_file.write("    label \"" + key + "\"" + "\n")
        output_file.write("  ]\n")

# print all edges
# print only those whose edge weight > 3
for keys in pairwise_counter_dictionary.keys():
    if (pairwise_counter_dictionary[keys[0], keys[1]]) >= MINIMUM_EDGE_WEIGHT:
        output_file.write("  edge\n")
        output_file.write("  [\n")
        output_file.write("    source " + str(keys[0]) + "\n")
        output_file.write("    target " + str(keys[1]) + "\n")
        output_file.write("    value " + str(pairwise_counter_dictionary[keys[0], keys[1]]) + "\n")
        output_file.write("  ]\n")

output_file.write("]\n")
output_file.close()

print("gml written")

gml written
